# Extract Raw Prices

This notebook demonstrates how to extract 15-minute prices from HDF5. There are 17 price points for each stock: 9:30 (open), 9:45, ..., 11:30, 13:15, ..., 14:45, 15:00 (close). The resulting dataframe for each stocks includes 18 columns: date string and the 17 price points.

In [1]:
import numpy as np
import pandas as pd
import math
import h5py
import zipfile
import os

### Extract from H5

In [49]:
####### See all stocks constained in H5 and their fields
with h5py.File('data_8raw.h5', 'r') as hf:
    keys = list(hf.keys())
    print("stocks included:")
    print(keys)
    print("stock {}'s data and shape:".format(keys[0]))
    print(hf[keys[0]][:])
    print(hf[keys[0]].shape)

stocks included:
['000006', '000029', '000031', '000923', '600128', '600158', '600679', '600884']
stock 000006's data and shape:
[( 91211.01, 3.668886 , 3.6773787, 3.660393 , 3.700026 , 3.6971953, 3.6887023, 3.6773787, 3.6858714, 3.6745477, 3.6802096, 3.663224 , 3.6434076, 3.6349146, 3.6490693, 3.6519003, 3.6547313, 3.6405766)
 ( 91214.01, 3.6405766, 3.5669725, 3.5386631, 3.5471559, 3.5386631, 3.5160155, 3.5301702, 3.5443249, 3.5301702, 3.5698032, 3.5698032, 3.5896199, 3.6179292, 3.62076  , 3.6150982, 3.5669725, 3.5924509)
 ( 91215.01, 3.5386631, 3.5103538, 3.4763827, 3.4707208, 3.462228 , 3.459397 , 3.4707208, 3.4735518, 3.4792135, 3.4820445, 3.459397 , 3.453735 , 3.4395804, 3.4282568, 3.4678898, 3.459397 , 3.4509041)
 ...
 (220907.02, 4.45     , 4.43     , 4.43     , 4.45     , 4.46     , 4.45     , 4.46     , 4.45     , 4.47     , 4.48     , 4.48     , 4.47     , 4.47     , 4.47     , 4.48     , 4.48     , 4.48     )
 (220908.02, 4.48     , 4.44     , 4.45     , 4.44     , 4.45     

In [45]:
####### Extract data from h5 and convert into one dataframe
def get_dict(file):
    '''Take an h5 file, outputs a dictionary with keys being stock code, values being the data'''
    
    result = {}
    
    with h5py.File(file, 'r') as hf:
        ks = list(hf.keys())
        for k in keys:
            ds = hf[k][:]
            values = np.zeros((ds.shape[0], len(list(ds[0]))), dtype='float64')
            for i in range(ds.shape[0]):
                values[i] = list(ds[i])
            result[k] = values
    
    return result

stocks8 = get_dict('data_8raw.h5')

def get_df_by_cols(stocks_dict, cols_ind, cols_name, ind= 'company'):
    
    df = pd.DataFrame()
    for k in stocks8.keys():
        cur_df = pd.DataFrame(np.array(stocks_dict[k][:, cols_ind]))
        cur_df[ind] = [k] * stocks_dict[k].shape[0]
        cur_df.set_index(ind, inplace= True)
        df = pd.concat([df, cur_df], axis= 0)
    df.columns = cols_name
    
    return df

cols_ind, cols_name = list(range(18)), ['date', 'open', '9:45', '10:00', '10:15', 
                                        '10:30', '10:45', '11:00', '11:15', '11:30', 
                                        '13:15', '13:30', '13:45', '14:00', '14:15',
                                        '14:30', '14:45', 'close']

df_8stocks = get_df_by_cols(stocks8, cols_ind, cols_name)
df_8stocks

,date,open,9:45,10:00,10:15,10:30,10:45,11:00,11:15,11:30,13:15,13:30,13:45,14:00,14:15,14:30,14:45,close
company,,,,,,,,,,,,,,,,,,
000006,91211.007812,3.668886,3.677379,3.660393,3.700026,3.697195,3.688702,3.677379,3.685871,3.674548,3.680210,3.663224,3.643408,3.634915,3.649069,3.651900,3.654731,3.640577
000006,91214.007812,3.640577,3.566972,3.538663,3.547156,3.538663,3.516016,3.530170,3.544325,3.530170,3.569803,3.569803,3.589620,3.617929,3.620760,3.615098,3.566972,3.592451
000006,91215.007812,3.538663,3.510354,3.476383,3.470721,3.462228,3.459397,3.470721,3.473552,3.479213,3.482044,3.459397,3.453735,3.439580,3.428257,3.467890,3.459397,3.450904
000006,91216.007812,3.450904,3.462228,3.470721,3.462228,3.445242,3.442411,3.425426,3.408440,3.411271,3.425426,3.431088,3.453735,3.450904,3.459397,3.453735,3.439580,3.436750
000006,91217.007812,3.456566,3.459397,3.459397,3.450904,3.439580,3.453735,3.450904,3.442411,3.425426,3.425426,3.439580,3.411271,3.425426,3.445242,3.482044,3.467890,3.465059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600884,220905.015625,24.650000,24.299999,24.730000,24.580000,24.469999,24.480000,24.299999,24.410000,24.459999,24.379999,24.379999,24.430000,24.350000,24.309999,24.400000,24.360001,24.480000
600884,220906.015625,24.549999,24.219999,24.190001,24.250000,24.450001,24.500000,24.730000,24.900000,24.930000,24.930000,24.940001,24.820000,24.879999,24.850000,24.900000,24.969999,25.000000
600884,220907.015625,24.820000,24.879999,25.180000,25.100000,25.410000,25.510000,25.379999,25.290001,25.240000,25.280001,25.240000,25.129999,25.110001,25.110001,25.150000,25.160000,25.150000


In [50]:
####### Extract each stock's data from the above dataframe

def clean_date(d):
    '''Convert date from float to string'''
    d = str(int(d))
    if len(d) == 6:
        d = '20' + d
    else:
        d = '200' + d
    return d[:4] + '-' + d[4:6] + '-' + d[6:]

def clean_single_stock(symbol):
    '''Create a dataframe containing data of the given stock id'''
    stock = df_8stocks.loc[symbol].reset_index()
    stock_cleaned = stock.iloc[14:, 1:]
    stock_cleaned['date'] = stock_cleaned['date'].apply(lambda x: clean_date(x))
    return stock_cleaned

stocks = ['000006', '000923', '600158', '600679']
data_list = []
for s in stocks:
    data = clean_single_stock(s)
    data_list.append(data)
    
data_list[0]

,date,open,9:45,10:00,10:15,10:30,10:45,11:00,11:15,11:30,13:15,13:30,13:45,14:00,14:15,14:30,14:45,close
14,2010-01-04,3.207444,3.187628,3.173473,3.179135,3.170642,3.184797,3.181966,3.173473,3.162149,3.164980,3.162149,3.156487,3.179135,3.170642,3.162149,3.156487,3.147995
15,2010-01-05,3.147995,3.097038,3.048912,3.051743,3.046081,3.046081,3.051743,3.077221,3.091376,3.091376,3.085714,3.060236,3.074390,3.065898,3.057405,3.054574,3.057405
16,2010-01-06,3.054574,3.080052,3.040419,3.057405,3.048912,3.065898,3.054574,3.111192,3.111192,3.088545,3.085714,3.071560,3.068729,3.060236,3.060236,3.057405,3.065898
17,2010-01-07,3.071560,3.051743,3.017772,3.012110,3.020603,3.026265,3.020603,3.080052,3.085714,3.074390,3.085714,3.082883,3.060236,3.060236,3.043250,3.029096,3.040419
18,2010-01-08,3.029096,3.071560,3.091376,3.074390,3.063067,3.046081,3.057405,3.040419,3.046081,3.048912,3.046081,3.046081,3.060236,3.074390,3.085714,3.097038,3.114023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2969,2022-09-05,4.320000,4.320000,4.340000,4.360000,4.360000,4.360000,4.370000,4.380000,4.370000,4.390000,4.380000,4.390000,4.370000,4.380000,4.390000,4.390000,4.390000
2970,2022-09-06,4.400000,4.430000,4.460000,4.470000,4.480000,4.480000,4.450000,4.460000,4.450000,4.480000,4.470000,4.460000,4.480000,4.470000,4.460000,4.470000,4.470000
2971,2022-09-07,4.450000,4.430000,4.430000,4.450000,4.460000,4.450000,4.460000,4.450000,4.470000,4.480000,4.480000,4.470000,4.470000,4.470000,4.480000,4.480000,4.480000
2972,2022-09-08,4.480000,4.440000,4.450000,4.440000,4.450000,4.450000,4.450000,4.440000,4.450000,4.440000,4.460000,4.450000,4.460000,4.450000,4.460000,4.470000,4.480000


In [51]:
####### Check if data contains duplicated dates
for d in data_list:
    print(d['date'].duplicated().any())

False
False
False
False


### Save to CSV

In [52]:
for i in range(len(data_list)):
    data_list[i].to_csv(stocks[i] + '_raw.csv', index= False)